In [4]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

from model import GentrificationModel
from mesa.batchrunner import BatchRunner

from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import numpy as np
import math

In [ ]:
# Define the model inputs
n = 3
problem = {
    'num_vars': n,
    'names': ['mobil', 'var', 'd_factor'],
    'bounds': [[0.001, 0.1],
               [0.001, 0.1],
               [0.000, 2.0]]
}

# Generate samples
param_values = saltelli.sample(problem, 200)

Y = np.zeros([param_values.shape[0]])

with tqdm(param_values.size, disable=False) as pbar:
    for i, X in enumerate(param_values):
        model = GentrificationModel(25, 25, 0.0015, X[0], 0.50, X[1], X[2])
        while model.running:
            model.step()
        Y[i] = model.gent_time
        pbar.update()


0it [00:00, ?it/s]

In [ ]:
#use feasible solutions
A = np.array([])
for i in range(0,Y.size,2*n+2):
    if not math.isnan(sum(Y[i:i+2*n+2])):
        A = np.append(A, Y[i:i+2*n+2])

In [ ]:
# Perform analysis
Si = sobol.analyze(problem, A, print_to_console=True)

# Print the first-order sensitivity indices
print(Si['S1'])